In [84]:
import torch
import torch.nn as nn

In [85]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # 词汇表大小
    "context_length": 1024,  # 上下文长度
    "emb_dim": 768,  # 嵌入维度
    "n_heads": 12,  # 注意力头数
    "n_layers": 12, #层数
    "drop_rate": 0.1,  # Dropout率
    "qkv_bias": False  # Query-Key-Value偏置
}

In [86]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out nust be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask", torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill(mask_bool, -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [87]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return (
            0.5
            * x
            * (
                1
                + torch.tanh(
                    torch.sqrt(torch.tensor(2.0 / torch.pi))
                    * (x + 0.044715 * torch.pow(x, 3))
                )
            )
        )

In [88]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [89]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [90]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg: GPT_CONFIG_124M):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"],
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        # 层归一化
        x = self.norm1(x)
        # 多头注意力
        x = self.att(x)
        # 丢弃部分样本
        x = self.drop_shortcut(x)
        # 残差连接
        x = x + shortcut
        # 对前网络模块的残差连接
        shortcut = x
        # 层归一化
        x = self.norm2(x)
        # 前馈神经网络
        x = self.ff(x)
        # 丢弃部分样本
        x = self.drop_shortcut(x)
        # 残差连接
        x = x + shortcut
        return x

In [91]:
torch.manual_seed(123)
# 创建形状为[batch_size,num_tokens,emb_dim]的样本输入
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Ouput shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Ouput shape: torch.Size([2, 4, 768])


In [92]:
class GPTModel(nn.Module):
    def __init__(self, cfg: GPT_CONFIG_124M):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        device = in_idx.device
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [93]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Evert effort moves you"
txt2 = "Evert day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
# 编码输入文本
batch = torch.stack(batch, dim=0)
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)

In [94]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters:{total_params:,}")
print("Token embedding layer shape:",model.tok_emb.weight.shape)
print("Output layer shape:",model.out_head.weight.shape)

Total number of parameters:163,009,536
Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [95]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(
    f"Number of trainable parameters "
    f"considering weight tying: {total_params_gpt2:,}"
)
# 计算总大小(假设float32,每个参数4字节)
total_size_bytes = total_params * 4
# 转换为兆字节
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Number of trainable parameters considering weight tying: 124,412,160
Total size of the model: 621.83 MB


In [96]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # 预测单词的模块
    # idx是当前上下文中的(batch,n_tokens)索引数组
    for _ in range(max_new_tokens):
        # 每次生成一个单词后,重新将其加入序列中
        # 如果当前上下文长度超过模型支持的最大上下文长度则截取
        # 例如,如果LLM只支持5个token,而上下文长度为10
        # 那么只使用最后5个token作为上下文
        idx_cond = idx[:, -context_size:]
        print(idx_cond)
        # 如果idx的长度超过模型支持的上下文长度size,只保留最后size个token
        # 避免溢出
        # 获取预测结果
        with torch.no_grad():  # 在推理阶段,不需要计算梯度,因为没有反向传播
            # 这样可以检查存储开销
            logits = model(idx_cond)
            print(logits.shape)
            # 模型输出结果
        # 只关注最后一个时间步的输出
        # (batch,n_tokens,vocab_size) 变为 (batch,vocab_size)
        logits = logits[:, -1, :]
        print(logits.shape)
        # 只关注最后一个时间步
        # 使用softmax函数计算概率
        # (batch,vocabsize)
        probas = torch.softmax(logits, dim=-1)
        # 归一化
        # 获取具有最高概率值的词汇索引
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        # 获取概率最高的词汇索引
        # 将采样的索引添加到序列中
        # (batch,n_tokens+1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [97]:
start_context = "Hello, I am"
# 模拟
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
# 进行语义理解
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [98]:
# 最终输出格式
# disable dropout
model.eval()
# 在检验的时候不需要正则化了
out = generate_text_simple(
    model=model,
    # 左边的参数名字,右边是函数传入的实际模型
    idx=encoded_tensor,  # 上下文的索引
    max_new_tokens=6,  # 最多运行六次,然后取结果概率最高的
    # 初始文本➕6
    context_size=GPT_CONFIG_124M["context_length"],
)

print("Output:", out)
print("Output length:", len(out[0]))
# 输出长度还有每个单词的id

tensor([[15496,    11,   314,   716]])
torch.Size([1, 4, 50257])
torch.Size([1, 50257])
tensor([[15496,    11,   314,   716, 27018]])
torch.Size([1, 5, 50257])
torch.Size([1, 50257])
tensor([[15496,    11,   314,   716, 27018,  7283]])
torch.Size([1, 6, 50257])
torch.Size([1, 50257])
tensor([[15496,    11,   314,   716, 27018,  7283, 46275]])
torch.Size([1, 7, 50257])
torch.Size([1, 50257])
tensor([[15496,    11,   314,   716, 27018,  7283, 46275, 41426]])
torch.Size([1, 8, 50257])
torch.Size([1, 50257])
tensor([[15496,    11,   314,   716, 27018,  7283, 46275, 41426, 33167]])
torch.Size([1, 9, 50257])
torch.Size([1, 50257])
Output: tensor([[15496,    11,   314,   716, 27018,  7283, 46275, 41426, 33167, 33239]])
Output length: 10


In [99]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Feature IT snowballProtect youngstersMu
